### Things to be clearified
>1. we use the Move system will have same access to the internet like on our own machine?
>2. If we have the possibility to download file from  the Move system?
>3. What's the point of using the MoVE system other than using a vm on own machine or ubuntu on our PC?

# FIT5148 - Distributed Databases and Big Data

# Assignment 1 - Solution Workbook


**Instructions:**
- You will be using Python 3.
- Read the assignment instruction carefully and implement the algorithms in this workbook. 
- You can use the datasets fireData and climateData (provided below) if you are aiming for Credit Task.
- For Distinction and High Distinction tasks, you are required to read the files FireData.csv and ClimateData.CSV provided with the assignment programatically and prepare the data in the correct format so that it can be used in your algorithm. 
- You can introduce new cells as necessary.

**Your details**
- Name: Boyu Zhang
- Student ID:28491300 

- Name:
- Student ID:

Let's get started!

In [27]:
# You can use the datasets fireData and climateData to implement your algorithms
climateData = []
fireData = []

In [28]:
print(climateData)
print(fireData)

[]
[]


In [29]:
import pandas as pd
#import multiprocessing as mp
import csv

In [30]:
firePath = './data/FireData.csv'
climatePath = './data/ClimateData.csv'
FireData = pd.read_csv('./data/FireData.csv')
ClimateData = pd.read_csv('./data/ClimateData.csv')

In [31]:
FireData.count()

Latitude                         2668
Longitude                        2668
Surface Temperature (kelvin)     2668
Datetime                         2668
Power                            2668
Confidence                       2668
Date                             2668
Surface Temperature (Celcius)    2668
dtype: int64

In [32]:
ClimateData.count()

Station                        366
 Date                          366
   Air Temperature(Celcius)    366
  Relative Humidity            366
  WindSpeed  (knots)           366
 Max Wind Speed                366
   MAX                         366
  MIN                          366
Precipitation                  366
dtype: int64

In [33]:
def read_to_list(path):
    with open(path,'r') as f:
        reader = csv.reader(f)
        return list(reader)


In [34]:
climateData = read_to_list(climatePath)
fireData = read_to_list(firePath)